# 3 - Plotting fields in Tokamak geometry

In this tutorial we will learn how to plot FEEC data in poloidal planes of a Tokamak geometry. 

## Time, logical and physical grids

Let us start by inspecting the content of the `post_processing/` folder, see [Tutorial 2](https://struphy.pages.mpcdf.de/struphy/doc/_build/html/tutorials/tutorial_02_postproc_standard_plotting.html):

In [ ]:
import os
import struphy

path_out = os.path.join(struphy.__path__[0], 'io/out/', 'tutorial_03/')
data_path = os.path.join(path_out, 'post_processing')

os.listdir(data_path)

Since this was a run of the model `LinearMHD`, only `fields_data/` is present in the output. Let us get the time grid,

In [ ]:
import numpy as np

t_grid = np.load(os.path.join(data_path, 't_grid.npy'))
t_grid

and inspect the `fields_data/`:

In [ ]:
fluid_path = os.path.join(data_path, 'fields_data')

print(os.listdir(fluid_path))

The MHD variables (i.e. their values at predefined grid points) have been saved in the folder `mhd/` (note that the magnetic field is stored under `em_fields/`). 

Before plotting we want some information on the geometry, initial conditions etc. For this, let us load the parameter file of the simulation:

In [ ]:
params_path = os.path.join(path_out, 'parameters.yml')

import yaml

with open(params_path) as file:
    parameters = yaml.load(file, Loader=yaml.FullLoader)

Let us look at the grid, geometry and fluid species:

In [ ]:
parameters['grid']

In [ ]:
parameters['geometry']

In [ ]:
parameters['fluid']

The simulation was done in `Tokamak` geometry with a resolution of `[8, 32, 4]`. In any toroidal geometry in Struphy, poloidal planes are always parametrized by the first two coordinates $(\eta_1, \eta_2)$ (here with a resolution `[8, 32]`), where $\eta_1$ parametrizes the radial coordinate and $\eta_2$ the angle coordinate. The toroidal angle is always parametrized by $\eta_3$ (here with a resolution of `[4]`). 

Moreover, we spot an MHD initial condition of type `TorusModesSin`, where the second (poloidal-angle) component of the MHD velocity `u2` is initialized in logical coordinates with a poloidal mode number $m=3$, toroidal mode number $n=1$ and an amlitude 0.001. The radial profile is given by a half-period `sin` function (zero at the edges).

Let us first load the logical 1d grid vectors:

In [ ]:
import pickle

with open(os.path.join(fluid_path, 'grids_log.bin'), 'rb') as file:
    e1_grid, e2_grid, e3_grid = pickle.load(file)
    
print('Shape of the e1_grid:', e1_grid.shape)
print('Shape of the e2_grid:', e2_grid.shape)
print('Shape of the e3_grid:', e3_grid.shape)

The **logical grid** is a product grid. In any one poloidal plane, the grid is a product of the `e1_grid` and `e2_grid` vectors: 

In [ ]:
from matplotlib import pyplot as plt

ee1, ee2 = np.meshgrid(e1_grid, e2_grid, indexing='ij')

for i in range(e1_grid.size):
    plt.plot(ee1[i, :], ee2[i, :], 'tab:blue', alpha=.5, zorder=0)
for j in range(e2_grid.size):
    plt.plot(ee1[:, j], ee2[:, j], 'tab:blue', alpha=.5, zorder=0)
plt.xlabel('$\eta_1$')
plt.ylabel('$\eta_2$')
plt.title(f'logical poloidal grid')
plt.axis('equal')

Let us now load the **physical grid** (mapped from the logical grid):

In [ ]:
with open(os.path.join(fluid_path, 'grids_phy.bin'), 'rb') as file:
    x_grid, y_grid, z_grid = pickle.load(file)
    
print('Shape of the x-grid:', x_grid.shape)
print('Shape of the y-grid:', y_grid.shape)
print('Shape of the z-grid:', z_grid.shape)

The physical grids are already stored as meshgrids, because they are not product grids.

In a Tokamak, we need to compute the $R$- and $\phi$-grids from the physical grid (please check out the [Struphy domain documentation](https://struphy.pages.mpcdf.de/struphy/sections/domains.html) for more details on the definition of the specific mappings). In Struphy, we have the convention that the **poloidal angle goes counter-clock-wise** (at toroidal $\phi = 0$) and the **toroidal angle goes clock-wise** when viewed from the top. 

The mapping $F_2: (R, \phi, Z) \mapsto (x, y, z)$ is thus given by
$$
\begin{aligned}
 x &= R \cos(\phi) \\
 y &= -R\sin(\phi) \\
 z &= Z
 \end{aligned}
$$

The Jacobian and its inverse are given by
$$
 DF_2 = \begin{pmatrix}
 \cos(\phi) & - R \sin(\phi) & 0
 \\
 -\sin(\phi) & - R \cos(\phi) & 0
 \\
 0 & 0 & 1
 \end{pmatrix}\,,\qquad
 DF_2^{-1} = \begin{pmatrix}
 \cos(\phi) & - \sin(\phi) & 0
 \\
 -\frac 1R \sin(\phi) & - \frac 1R \cos(\phi) & 0
 \\
 0 & 0 & 1
 \end{pmatrix}\,.
$$
The Jacobian determinant and metric tensor read
$$
 \textnormal{det} DF_2 = - R\,,\qquad G = \begin{pmatrix}
 1 & 0 & 0
 \\
 0 &  R^2 & 0
 \\
 0 & 0 & 1
 \end{pmatrix}\,.
$$


We must pay attention to the parameter `tor_period` of the [Tokamak](https://struphy.pages.mpcdf.de/struphy/sections/domains.html#struphy.geometry.domains.Tokamak) geometry, which was set to 3, meaning that only one third of the total Torus has been simulated, $\phi \in (0, 2\pi/3)$.

In [ ]:
R_grid = np.sqrt(x_grid**2 + y_grid**2)
phi_grid = np.arctan2(-y_grid, x_grid)

for n in range(phi_grid.shape[2]):
    print(phi_grid[0, 0, n])
    
print('check: 2pi/3 = ', 2*np.pi/3)

Let us now plot the grids of the five available poloidal planes. Since the geometry was a [Tokamak](https://struphy.pages.mpcdf.de/struphy/sections/domains.html#struphy.geometry.domains.Tokamak), they must be identical:

In [ ]:
from matplotlib import pyplot as plt

n_poloidal_planes = e3_grid.size

plt.figure(figsize=(12, 16))
for n in range(n_poloidal_planes):
    plt.subplot(3, 2, n + 1)
    for i in range(R_grid.shape[0]):
        plt.plot(R_grid[i, :, n], z_grid[i, :, n], 'tab:blue', alpha=.5, zorder=0)
    for j in range(R_grid.shape[1] - 1):
        plt.plot(R_grid[:, j, n], z_grid[:, j, n], 'tab:blue', alpha=.5, zorder=0)
    plt.xlabel('R')
    plt.ylabel('Z')
    plt.title(f'$\phi = {phi_grid[0, 0, n]}$')
    plt.axis('equal')

## MHD velocity plots

In the simulation for this notebook, the MHD velocity was in the space $u \in (H^1(\Omega))^3$ representing vector-fields (contra-variant representation). 

We start by loaging the values of $u$ in logical space:

In [ ]:
with open(os.path.join(fluid_path, 'mhd/', 'u2_log.bin'), 'rb') as file:
    u_log = pickle.load(file)

for key, val in u_log.items():
    print(key)
    for va in val:
        print(va.shape)

Let us plot the initial condition of the second comonent of $u$ in the **logical poloidal planes** ($\eta_1$-$\eta_2$):

In [ ]:
u_log_init = u_log[0.0]

plt.figure(figsize=(12, 16))
for n in range(n_poloidal_planes):
    plt.subplot(3, 2, n + 1)
    plt.contourf(e1_grid, e2_grid, u_log_init[1][:, :, n].T)
    plt.xlabel('$\eta_1$')
    plt.ylabel('$\eta_2$')
    plt.title(f'$\hat u_2(t=0)$ at $\eta_3={e3_grid[n]}$')
    plt.colorbar()

We can see the initial condition with a poloidal mode number $m=3$ (in $\eta_2$), a toroidal mode number $n=1$ (in $\eta_3$) and the half-sine function in the radial $\eta_1$ direction. The amplitude of 0.001 also corresponds to the value specified in the parameter file.

The component $\hat u_2$ is the contra-variant component (because $u \in (H^1(\Omega))^3$) along the angle-direction $\eta_2$, which is the poloidal angle of a torus. Let us plot the same initial conditions as above in the physical domain:

In [ ]:
plt.figure(figsize=(12, 16))
for n in range(n_poloidal_planes):
    plt.subplot(3, 2, n + 1)
    plt.contourf(R_grid[:, :, n], z_grid[:, :, n], u_log_init[1][:, :, n])
    plt.colorbar()
    plt.title(f'$\hat u_2(t=0)$ at $\phi={phi_grid[0, 0, n]}$')
    plt.xlabel('R')
    plt.ylabel('Z')
    plt.axis('equal')

Moreover, we can plot the vector components in different coordinates. For instance, let us plot $u^*_R$, i.e. the R-component of the vector field in $(R, \phi,Z)$-cooridnates, denoted with a star.

 For this we need to pullback the physical $(x, y, z)$-components of $u$ to $(R, \phi, Z)$-space. The pullback is given by the inverse Jacobian,
$$
 u^* = DF_2^{-1} u\,.
$$

For the first ($R$-)component this means
$$
 u^*_R = \cos(\phi) u_x - \sin(\phi) u_y\,.
$$

We now load the values of $u$ in the physical domain:

In [ ]:
with open(os.path.join(fluid_path, 'mhd/', 'u2_phy.bin'), 'rb') as file:
    u_phy = pickle.load(file)

for key, val in u_phy.items():
    print(key)
    for va in val:
        print(va.shape)

Let us plot the initial condition of $u^*_R$ in the five poloidal planes:

In [ ]:
u_phy_init = u_phy[0.0]

plt.figure(figsize=(12, 16))
for n in range(n_poloidal_planes):
    
    uR = np.cos(phi_grid[:, :, n]) * u_phy_init[0][:, :, n] - np.sin(phi_grid[:, :, n]) * u_phy_init[1][:, :, n]
    
    plt.subplot(3, 2, n + 1)
    plt.contourf(R_grid[:, :, n], z_grid[:, :, n], uR)
    plt.xlabel('$R$')
    plt.ylabel('$Z$')
    plt.title(f'$u^*_R(t=0)$ at $\phi={phi_grid[0, 0, n]}$')
    plt.colorbar()
    plt.axis('equal')

We can also show the vector field $(u^*_R, u^*_Z)$ in the poiloidal $(R,Z)$-plane:

In [ ]:
plt.figure(figsize=(12, 16))
for n in range(n_poloidal_planes):
    
    uR = np.cos(phi_grid[:, :, n]) * u_phy_init[0][:, :, n] - np.sin(phi_grid[:, :, n]) * u_phy_init[1][:, :, n]
    uZ = u_phy_init[2][:, :, n]
    
    plt.subplot(3, 2, n + 1)
    plt.contourf(R_grid[:, :, n], z_grid[:, :, n], np.sqrt(uR**2 + uZ**2))
    plt.quiver(R_grid[:, :, n], z_grid[:, :, n], uR, uZ, scale=.03, color='w')
    plt.xlabel('$R$')
    plt.ylabel('$Z$')
    plt.title(f'$u^*_R(t=0)$ at $\phi={phi_grid[0, 0, n]}$')
    plt.colorbar()
    plt.axis('equal')

## Time evolution 

Finally, let us plot the time evolution of the poloidal component $\hat u_2$ on the physical domain, at $\phi=0$:

In [ ]:
plt.figure(figsize=(12, 16))

for n, (t, u_comps) in enumerate(u_log.items()): 
    plt.subplot(3, 2, n + 1)
    plt.contourf(R_grid[:, :, 0], z_grid[:, :, 0], u_comps[1][:, :, 0])
    plt.colorbar()
    plt.title(f'$\hat u_2(R, 0, Z)$ at $\phi=0$ at t={t}')
    plt.xlabel('R')
    plt.ylabel('Z')
    plt.axis('equal')

As mentioned before, the velocity is here represented in the space $(H^1(\Omega))^3$, which in Struphy is the space for "vector fields" (=contra-variant components). We can also plot the co-variant components (1-form) of the velocity. 

For this, we need to recreate the domain used in the simulation and transform vector-field to 1-form components with  the `domain.transform` method: 

In [ ]:
from struphy.geometry.domains import Tokamak

params_map = parameters['geometry']['Tokamak']

domain = Tokamak(**params_map)

plt.figure(figsize=(12, 16))

for n, (t, u_comps) in enumerate(u_log.items()): 
    
    u1 = domain.transform(u_comps, e1_grid, e2_grid, e3_grid, kind='v_to_1')
    
    plt.subplot(3, 2, n + 1)
    plt.contourf(R_grid[:, :, 0], z_grid[:, :, 0], u1[1][:, :, 0])
    plt.colorbar()
    plt.title(f'1-form comp. $\hat u^1_2(R, 0, Z)$ at t={t}')
    plt.xlabel('R')
    plt.ylabel('Z')
    plt.axis('equal')